# 2.0 Resample, Pattern Matching and Feature Engineering

# Contents

- [1.0 Load Data](#1.0-Load-Data)
- [2.0 Resample Data](#2.0-Resample-Data)
    - [2.1 Monthly Data](#2.1-Monthly-Data)
    - [2.2 Daily Data](#2.2-Daily-Data)
    - [2.3 4-Hour Data](#2.3-4-Hour-Data)
- [3.0 Pattern Matching](#3.0-Pattern-Matching)
    - [3.1 Monthly Data](#3.1-Monthly-Data)
    - [3.2 Daily Data](#3.2-Daily-Data)
    - [3.3 4-Hour Data](#3.3-4-Hour-Data)


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import calendar
import plotly.graph_objects as go


In [2]:
pd.set_option('display.max_columns', None)

---

# 1.0 Load Data

In [3]:
eur_usd = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/csv/eur_usd.csv', 
                      index_col='date', parse_dates=True)

In [4]:
# eur_usd['day_name'] = eur_usd.index.day_name()
eur_usd = eur_usd[['open','high', 'low','close']]

In [5]:
eur_usd.head()

,open,high,low,close
date,,,,
2000-05-30 17:27:00,0.9302,0.9302,0.9302,0.9302
2000-05-31 00:50:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:51:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:55:00,0.9317,0.9317,0.9317,0.9317
2000-05-31 01:01:00,0.9318,0.9318,0.9318,0.9318


In [6]:
eur_usd.isnull().sum()

open     0
high     0
low      0
close    0
dtype: int64

In [7]:
# eur_usd['wday'] = eur_usd.index.day_name()

In [8]:
eur_usd.shape

(6539011, 4)

In [9]:
eur_usd.head()

,open,high,low,close
date,,,,
2000-05-30 17:27:00,0.9302,0.9302,0.9302,0.9302
2000-05-31 00:50:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:51:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:55:00,0.9317,0.9317,0.9317,0.9317
2000-05-31 01:01:00,0.9318,0.9318,0.9318,0.9318


In [10]:
eur_usd.loc[eur_usd.index == '2001-01-9']

,open,high,low,close
date,,,,
2001-01-09,0.9447,0.9447,0.9446,0.9446


---

## Holidays

In [11]:
holidays = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/EUR-USD Holidays/EUR-USD_Holidays.csv',
                   index_col='Date', parse_dates=True)

In [12]:
holidays.head()

,Reason
Date,
2012-01-05,Missing value; holiday or weekend (H)
2012-09-04,Missing value; holiday or weekend (H)
2012-06-04,Missing value; holiday or weekend (H)
2011-12-26,Missing value; holiday or weekend (H)
2011-04-25,Missing value; holiday or weekend (H)


In [13]:
holidays.isnull().sum()

Reason    0
dtype: int64

---

## Gold

In [14]:
gold = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/gold/gold_clean.csv',
                   index_col='date', parse_dates=True)

In [17]:
eur_usd.head()

,open,high,low,close
date,,,,
2000-05-30 17:27:00,0.9302,0.9302,0.9302,0.9302
2000-05-31 00:50:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:51:00,0.9315,0.9315,0.9315,0.9315
2000-05-31 00:55:00,0.9317,0.9317,0.9317,0.9317
2000-05-31 01:01:00,0.9318,0.9318,0.9318,0.9318


In [18]:
gold.head()

,usd,euro
date,,
2000-05-30,94.9,119.5
2000-05-31,94.6,119.8
2000-06-01,94.7,119.4
2000-06-02,97.9,121.7
2000-06-05,98.4,122.4


In [19]:
gold.rename(columns={"usd":"gold_usd","euro":"gold_euro"}, inplace=True)

In [20]:
eur_usd.loc[eur_usd.index[-1]].name

Timestamp('2019-12-31 16:59:00')

In [21]:
gold = gold.loc[gold.index <= eur_usd.loc[eur_usd.index[-1]].name]

In [22]:
gold.head()

,gold_usd,gold_euro
date,,
2000-05-30,94.9,119.5
2000-05-31,94.6,119.8
2000-06-01,94.7,119.4
2000-06-02,97.9,121.7
2000-06-05,98.4,122.4


In [25]:
gold.tail()

,gold_usd,gold_euro
date,,
2019-12-25,515.0,545.3
2019-12-26,515.0,544.9
2019-12-27,525.2,552.5
2019-12-30,526.3,551.7
2019-12-31,526.3,550.6


In [26]:
eur_usd.tail()

,open,high,low,close
date,,,,
2019-12-31 16:55:00,1.12117,1.12118,1.12115,1.12118
2019-12-31 16:56:00,1.12118,1.12120,1.12115,1.12115
2019-12-31 16:57:00,1.12115,1.12115,1.12105,1.12105
2019-12-31 16:58:00,1.12105,1.12110,1.12099,1.12099
2019-12-31 16:59:00,1.12099,1.12115,1.12076,1.12076


In [23]:
gold.shape

(5111, 2)

In [24]:
eur_usd.shape

(6539011, 4)

---

# 2.1 Monthly Data

In [ ]:
monthly = eur_usd.resample('m').agg({'open':'first','high':'max',
                                    'low':'min', 'close':'last'})
monthly.head(14)

In [ ]:
monthly.shape

In [ ]:
monthly.isnull().sum()

In [ ]:
# monthly.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/monthly.csv', index=True)

---

# 2.2 Daily Data

In [48]:
# create the super class for pattern identification

class Process_Market_Data(object):
    
    # Initialise parent class
    def __init__(self, market_data, data_name='eur-usd', time_frame='daily', 
                path_to_save_md='/Users/stuartdaw/Documents/Capstone_data/data/resampled/',
                path_to_save_ptn= '/Users/stuartdaw/Documents/Capstone_data/data/targets/',
                file_type='.csv', add_gold=True, 
                gold_file='/Users/stuartdaw/Documents/Capstone_data/data/gold/gold_clean.csv'):
        self.market_data = market_data
        self.data_name = data_name
        self.time_frame = time_frame
        self.path_to_save_md = path_to_save_md
        self.path_to_save_ptn = path_to_save_ptn
        self.file_type = file_type
        self.add_gold = add_gold
        self.gold_file = gold_file
        
    # Function to drive the other methods
    def process_data(self):
        # Get the correct time code
        self.get_time_code()
        self.resample_data()
        self.create_daily_mid_point()
        self.create_moving_avgs()
        self.remove_nans()
        self.add_vol()
        self.add_percent_change()
        self.add_height_info()
        self.add_height_of_prior_3_periods()
        self.apply_time_frame_direction()
        self.add_gold_data()
        self.remove_nans()
        
        
    def get_time_code(self):
        if self.time_frame == 'daily':
            # 'B' is for business days
            self.time_code = 'B'
            
    # Create Daily Data and remove weekend as doesnt have same liquidity and not 
    # going to use to form patterns
    def resample_data(self):
        self.market_data = self.market_data.resample(self.time_code).agg({'open':'first','high':'max',
                                                                            'low':'min','close':'last',})
    
    def create_daily_mid_point(self):
        self.market_data['mid'] = (self.market_data['close']+self.market_data['open'])/2
    
    def create_moving_avgs(self):
        if self.time_frame == 'monthly':
            self.market_data['bi_mnth_mv_avg'] = self.market_data['mid'].rolling(2).mean()
            self.market_data['qtr_mv_avg'] = self.market_data['mid'].rolling(3).mean()
        elif self.time_frame == 'daily':
            self.market_data['wk_mv_avg'] = self.market_data['mid'].rolling(5).mean()
            self.market_data['mnth_mv_avg'] = self.market_data['mid'].rolling(21).mean()
    
    def add_vol(self):
        if self.time_frame == 'daily':
            self.market_data['volatility_3_day'] = self.market_data['mid'].pct_change().rolling(3).std()
            self.market_data['volatility_10_day'] = self.market_data['mid'].pct_change().rolling(10).std()
        elif self.time_frame == 'monthly':
            self.market_data['volatility_2_mnth'] = self.market_data['mid'].pct_change().rolling(2).std()
            self.market_data['volatility_qtr'] = self.market_data['mid'].pct_change().rolling(3).std()
    
    def add_percent_change(self):
        self.market_data['pct_chge_3_prds'] = self.market_data['mid'].pct_change(3)
        self.market_data['pct_chge_5_prds'] = self.market_data['mid'].pct_change(5)
        self.market_data['pct_chge_10_prds'] = self.market_data['mid'].pct_change(10)
    
    def add_height_info(self):
        self.market_data['height'] = abs(self.market_data['close'] - self.market_data['open'])
            
    def add_height_of_prior_3_periods(self):
        self.market_data['height-1'] = abs(self.market_data['height'].shift(1))
        self.market_data['height-2'] = abs(self.market_data['height'].shift(2))
        self.market_data['height-3'] = abs(self.market_data['height'].shift(3))
        
    def time_frame_directon(self, row):
        val = 0
        if(row['open'] > row['close']):
            val = -1
        elif (row['open'] < row['close']):
            val = 1
        return val
                
    def apply_time_frame_direction(self):
        self.market_data['direction'] = self.market_data.apply(self.time_frame_directon, axis=1)

    def add_gold_data(self):
        self.gold = pd.read_csv(self.gold_file, index_col='date', parse_dates=True)
        self.gold.rename(columns={"usd":"gold_usd","euro":"gold_euro"}, inplace=True)
        self.gold = self.gold.loc[self.gold.index <= self.market_data.loc[self.market_data.index[-1]].name]
        self.market_data = pd.concat([self.market_data, self.gold], axis=1)
        
    # remove nulls due to moving averages
    def remove_nans(self):
        self.market_data.dropna(inplace=True)
    
    def print_pattern_chart(self):
        
        fig = go.Figure(data=[go.Candlestick(x=self.features.index,
                open=self.features['open'],
                high=self.features['high'],
                low=self.features['low'],
                close=self.features['close'])])

        fig.add_trace(go.Line(x=self.market_data.index, y=self.market_data['mid'], 
                              name='average', line=dict(color='black', width=0.1, dash='dot')))

        fig.update_layout(xaxis_rangeslider_visible=False)

        fig.show()
    
    def create_pattern_list(self):
        self.dates = pd.DataFrame(self.market_data.loc[self.market_data['select'] == 1].index)
        self.dates.rename(columns={'date':'pattern_end'}, inplace=True);

    def print_pattern_list(self):
        print(f"Len: {len(self.dates)}\n\nlist:\n{self.dates}")
        
    def create_csv_market(self):
        self.market_data.to_csv(self.path_to_save_md + self.data_name+ '2' + self.time_frame 
                                + self.file_type, index=True)
        

In [108]:
class Marubozu(Process_Market_Data):
    
    # Initialise parent class
    def __init__(self, market_data, data_name='eur-usd', time_frame='daily', 
                path_to_save_md='/Users/stuartdaw/Documents/Capstone_data/data/resampled/',
                path_to_save_ptn= '/Users/stuartdaw/Documents/Capstone_data/data/targets/',
                file_type='.csv', add_gold=True, 
                gold_file='/Users/stuartdaw/Documents/Capstone_data/data/gold/gold_clean.csv',
                pattern='Marabozu', min_height=0.005, target_factor=1 , test_timeframe=5,
                margin=0.0005):
        super().__init__(market_data, data_name='eur-usd', time_frame='daily', 
                path_to_save_md='/Users/stuartdaw/Documents/Capstone_data/data/resampled/', 
                path_to_save_ptn= '/Users/stuartdaw/Documents/Capstone_data/data/targets/',
                file_type='.csv', add_gold=True, 
                gold_file='/Users/stuartdaw/Documents/Capstone_data/data/gold/gold_clean.csv')
        self.pattern = pattern
        self.min_height = min_height
        self.target_factor = target_factor
        self.test_timeframe = test_timeframe
        self.margin = margin
        self.features = pd.DataFrame()
        self.maru_dict = {'A':0,'B':0,'C':0,'D':0}
        self.select_res_dict = {'A':0,'B':0,'C':0,'D':0,'E':0}

    def generate_pattern_list(self):
        self.process_data()
        self.classify_candles()
        self.add_other_marubozu()
        self.add_previous_days()
        self.add_future_prices()
        self.add_exit_price()
        self.choose_pattern_rows()
        self.choose_best_price()
        self.add_future_date()
        self.remove_nans()
        return self.market_data

        
    
    # Is Marubozu? Look for signicant candlesticks and their direction
    def marubozu_type(self, row):
    
        # Check its a significant height
        if abs(row['open'] - row['close']) < self.min_height:
            self.maru_dict['A'] +=1
            return 0

        # Check the direction
        if row['open'] > row['close']:
            self.maru_dict['B'] +=1
            return -1
        elif row['open'] < row['close']:
            self.maru_dict['C'] +=1
            return 1
        
            self.maru_dict['D'] +=1
        return 0

    # Tag the line with information on the candles
    def classify_candles(self):
        print(self.min_height)
        self.market_data['marubozu'] = self.market_data.apply(lambda row: self.marubozu_type(row), axis=1)
        
    def add_other_marubozu(self):
        self.market_data['marubozu+1'] = self.market_data['marubozu'].shift(-1)
        self.market_data['marubozu-1'] = self.market_data['marubozu'].shift(1)
        self.market_data['marubozu-2'] = self.market_data['marubozu'].shift(2)

    def add_previous_days(self):
        self.market_data['day-1_open'] = self.market_data['open'].shift(1)
        self.market_data['day-2_open'] = self.market_data['open'].shift(2)
        self.market_data['day-3_open'] = self.market_data['open'].shift(3)

        self.market_data['day-1_high'] = self.market_data['high'].shift(1)
        self.market_data['day-2_high'] = self.market_data['high'].shift(2)
        self.market_data['day-3_high'] = self.market_data['high'].shift(3)

        self.market_data['day-1_low'] = self.market_data['low'].shift(1)
        self.market_data['day-2_low'] = self.market_data['low'].shift(2)
        self.market_data['day-3_low'] = self.market_data['low'].shift(3)

        self.market_data['day-1_close'] = self.market_data['close'].shift(1)
        self.market_data['day-2_close'] = self.market_data['close'].shift(2)
        self.market_data['day-3_close'] = self.market_data['close'].shift(3)
        
        
    # create future columns for target and matching connected Marubuzo
    def add_future_prices(self):
        self.market_data['day+1_open'] = self.market_data['open'].shift(-1)
        self.market_data['day+1_high'] = self.market_data['high'].shift(-1)
        self.market_data['day+1_low'] = self.market_data['low'].shift(-1)
        self.market_data['day+1_close'] = self.market_data['close'].shift(-1)
        self.market_data['day+2_high'] = self.market_data['high'].shift(-2)
        self.market_data['day+2_low'] = self.market_data['low'].shift(-2)
        self.market_data['day+3_high'] = self.market_data['high'].shift(-3)
        self.market_data['day+3_low'] = self.market_data['low'].shift(-3)
        self.market_data['day+4_high'] = self.market_data['high'].shift(-4)
        self.market_data['day+4_low'] = self.market_data['low'].shift(-4)
        self.market_data['day+5_high'] = self.market_data['high'].shift(-5)        
        self.market_data['day+5_low'] = self.market_data['low'].shift(-5)        
        
    def choose_exit_price(self, row):
        return row['close'] + (row['height'] * row['direction'] * self.target_factor)

    def add_exit_price(self):
        self.market_data['exit_price'] = self.market_data.apply(lambda row: self.choose_exit_price(row), axis=1)
    
    # Select rows that have a opposite pattern of very similar height.
    def tag_rows(self, row):
        
        margin = self.margin

        if abs(row['marubozu']) != 1:
            self.select_res_dict['A'] +=1
            return 0

#         if ((row['marubozu-2'] == 1 and row['marubozu-1'] == -1) or (
#                 row['marubozu-2'] == -1 and row['marubozu-1'] == 1)) and (
#             ((abs(row['day-2_open'] - row['day-1_close']) < margin) and
#             (abs(row['day-2_close'] - row['day-1_open']) < margin))):
#             self.select_res_dict['B'] +=1    
#             return 0

        if  ((row['marubozu-1'] == 1 and row['marubozu'] == -1) or (
                row['marubozu-1'] == -1 and row['marubozu'] == 1)) and (
            ((abs(row['open'] - row['day-1_close']) < margin) and
            (abs(row['close'] - row['day-1_open']) < margin))):
            self.select_res_dict['C'] +=1
            return 1

        if ((row['marubozu+1'] == 1 and row['marubozu'] == -1) or (
                row['marubozu+1'] == -1 and row['marubozu'] == 1)) and (
            ((abs(row['open'] - row['day+1_close']) < margin) and
            (abs(row['close'] - row['day+1_open']) < margin))):
            self.select_res_dict['D'] +=1
            return 2 
        
        self.select_res_dict['E'] +=1
        return 0

    def choose_pattern_rows(self):
        self.market_data['select'] = self.market_data.apply(lambda row: self.tag_rows(row), axis=1)
        
    def get_best_price(self, row):
        if row['direction'] == 1:     
            return row[['day+1_high','day+2_high','day+3_high','day+4_high','day+5_high']].max()
        elif row['direction'] == -1:                 
            return row[['day+1_low','day+2_low','day+3_low','day+4_low','day+5_low']].min()
            
    def choose_best_price(self):
        self.market_data['target'] = self.market_data.apply(lambda row: self.get_best_price(row), axis=1)

        # Calculate the date of 5 business days ahead
        # update using fing loc
    def add_future_date(self):
        self.market_data['date+5'] = self.market_data.index.shift(self.test_timeframe, freq=self.time_code)
        
    def select_pattern(self):
        self.features = self.market_data.loc[(self.market_data['select'] == 1) | (self.market_data['select'] == 2)]
        self.create_pattern_list()
        
    def create_mlr_features(self):
        self.market_data.
        
    def print_marubozu_chart(self):
        self.select_pattern()
        self.print_pattern_chart()
        
    def create_csv_pattern(self):
        self.dates.to_csv(self.path_to_save_ptn + self.time_frame + '_pattern2' + self.file_type, index=False)

In [109]:
mb = Marubozu(eur_usd)

In [110]:
daily = mb.generate_pattern_list()

0.005


In [111]:
daily.loc[(daily.index > '2000-10-6') & (daily.index < '2000-10-18')]

,open,high,low,close,mid,wk_mv_avg,mnth_mv_avg,volatility_3_day,volatility_10_day,pct_chge_3_prds,pct_chge_5_prds,pct_chge_10_prds,height,height-1,height-2,height-3,direction,gold_usd,gold_euro,marubozu,marubozu+1,marubozu-1,marubozu-2,day-1_open,day-2_open,day-3_open,day-1_high,day-2_high,day-3_high,day-1_low,day-2_low,day-3_low,day-1_close,day-2_close,day-3_close,day+1_open,day+1_high,day+1_low,day+1_close,day+2_high,day+2_low,day+3_high,day+3_low,day+4_high,day+4_low,day+5_high,day+5_low,exit_price,select,target,date+5
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-09,0.8699,0.8710,0.8665,0.8680,0.86895,0.87299,0.868400,0.001566,0.002500,-0.008161,-0.012781,-0.010871,0.0019,0.0003,0.0043,0.0030,-1.0,93.8,126.8,0,1.0,0.0,0.0,0.8701,0.8745,0.8776,0.8743,0.8790,0.8780,0.8674,0.8672,0.8719,0.8698,0.8702,0.8746,0.8679,0.8747,0.8670,0.8732,0.8764,0.8644,0.8682,0.8597,0.8660,0.8530,0.8557,0.8462,0.8661,0,0.8462,2000-10-16
2000-10-10,0.8679,0.8747,0.8670,0.8732,0.87055,0.87158,0.868745,0.002331,0.002619,-0.002063,-0.008033,-0.009670,0.0053,0.0019,0.0003,0.0043,1.0,94.0,127.1,1,-1.0,0.0,0.0,0.8699,0.8701,0.8745,0.8710,0.8743,0.8790,0.8665,0.8674,0.8672,0.8680,0.8698,0.8702,0.8733,0.8764,0.8644,0.8677,0.8682,0.8597,0.8660,0.8530,0.8557,0.8462,0.8565,0.8480,0.8785,2,0.8764,2000-10-17
2000-10-11,0.8733,0.8764,0.8644,0.8677,0.87050,0.87046,0.869214,0.001513,0.001716,0.000632,-0.006392,-0.014491,0.0056,0.0053,0.0019,0.0003,-1.0,94.8,127.4,-1,0.0,1.0,0.0,0.8679,0.8699,0.8701,0.8747,0.8710,0.8743,0.8670,0.8665,0.8674,0.8732,0.8680,0.8698,0.8676,0.8682,0.8597,0.8628,0.8660,0.8530,0.8557,0.8462,0.8565,0.8480,0.8579,0.8327,0.8621,1,0.8327,2000-10-18
2000-10-12,0.8676,0.8682,0.8597,0.8628,0.86520,0.86903,0.869412,0.004140,0.002235,-0.004316,-0.008196,-0.019603,0.0048,0.0056,0.0053,0.0019,-1.0,95.2,129.6,0,-1.0,-1.0,1.0,0.8733,0.8679,0.8699,0.8764,0.8747,0.8710,0.8644,0.8670,0.8665,0.8677,0.8732,0.8680,0.8627,0.8660,0.8530,0.8554,0.8557,0.8462,0.8565,0.8480,0.8579,0.8327,0.8487,0.8383,0.8580,0,0.8327,2000-10-19
2000-10-13,0.8627,0.8660,0.8530,0.8554,0.85905,0.86685,0.869279,0.003811,0.002683,-0.013210,-0.012529,-0.026076,0.0073,0.0048,0.0056,0.0053,-1.0,94.9,129.7,-1,-1.0,0.0,-1.0,0.8676,0.8733,0.8679,0.8682,0.8764,0.8747,0.8597,0.8644,0.8670,0.8628,0.8677,0.8732,0.8553,0.8557,0.8462,0.8491,0.8565,0.8480,0.8579,0.8327,0.8487,0.8383,0.8475,0.8351,0.8481,0,0.8327,2000-10-20
2000-10-16,0.8553,0.8557,0.8462,0.8491,0.85220,0.86350,0.868919,0.000944,0.003154,-0.021022,-0.019276,-0.031811,0.0062,0.0073,0.0048,0.0056,-1.0,94.4,130.6,-1,1.0,-1.0,0.0,0.8627,0.8676,0.8733,0.8660,0.8682,0.8764,0.8530,0.8597,0.8644,0.8554,0.8628,0.8677,0.8493,0.8565,0.8480,0.8546,0.8579,0.8327,0.8487,0.8383,0.8475,0.8351,0.8426,0.8345,0.8429,0,0.8327,2000-10-23
2000-10-17,0.8493,0.8565,0.8480,0.8546,0.85195,0.85978,0.868817,0.004207,0.003289,-0.015314,-0.021366,-0.029227,0.0053,0.0062,0.0073,0.0048,1.0,93.9,129.7,1,-1.0,-1.0,-1.0,0.8553,0.8627,0.8676,0.8557,0.8660,0.8682,0.8462,0.8530,0.8597,0.8491,0.8554,0.8628,0.8546,0.8579,0.8327,0.8400,0.8487,0.8383,0.8475,0.8351,0.8426,0.8345,0.8400,0.8349,0.8599,0,0.8579,2000-10-24


In [112]:
mb.select_res_dict

{'A': 2582, 'B': 0, 'C': 64, 'D': 62, 'E': 1933}

In [113]:
mb.maru_dict

{'A': 2582, 'B': 1014, 'C': 1045, 'D': 0}

In [114]:
mb.market_data.loc[(mb.market_data.index > '2001-04-03') & (mb.market_data.index < '2001-04-11')]

,open,high,low,close,mid,wk_mv_avg,mnth_mv_avg,volatility_3_day,volatility_10_day,pct_chge_3_prds,pct_chge_5_prds,pct_chge_10_prds,height,height-1,height-2,height-3,direction,gold_usd,gold_euro,marubozu,marubozu+1,marubozu-1,marubozu-2,day-1_open,day-2_open,day-3_open,day-1_high,day-2_high,day-3_high,day-1_low,day-2_low,day-3_low,day-1_close,day-2_close,day-3_close,day+1_open,day+1_high,day+1_low,day+1_close,day+2_high,day+2_low,day+3_high,day+3_low,day+4_high,day+4_low,day+5_high,day+5_low,exit_price,select,target,date+5
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-04-04,0.8966,0.9080,0.8937,0.9066,0.90160,0.88668,0.903088,0.006009,0.008144,0.026236,0.012465,-0.001550,0.0100,0.0138,0.0054,0.0019,1.0,90.2,117.3,1,-1.0,1.0,1.0,0.8829,0.8775,0.8795,0.8987,0.8837,0.8865,0.8821,0.8745,0.8755,0.8967,0.8829,0.8776,0.9066,0.9091,0.8943,0.8967,0.9051,0.8927,0.9051,0.8947,0.8995,0.8868,0.8918,0.8814,0.9166,2,0.9091,2001-04-11
2001-04-05,0.9066,0.9091,0.8943,0.8967,0.90165,0.89036,0.901571,0.007044,0.006930,0.024369,0.020832,0.010988,0.0099,0.0100,0.0138,0.0054,-1.0,89.7,117.5,-1,1.0,1.0,1.0,0.8966,0.8829,0.8775,0.9080,0.8987,0.8837,0.8937,0.8821,0.8745,0.9066,0.8967,0.8829,0.8966,0.9051,0.8927,0.9034,0.9051,0.8947,0.8995,0.8868,0.8918,0.8814,0.8946,0.8840,0.8868,1,0.8814,2001-04-12
2001-04-06,0.8966,0.9051,0.8927,0.9034,0.90000,0.89465,0.900050,0.008223,0.006893,0.011463,0.024415,0.011691,0.0068,0.0099,0.0100,0.0138,1.0,89.9,117.2,1,-1.0,-1.0,1.0,0.9066,0.8966,0.8829,0.9091,0.9080,0.8987,0.8943,0.8937,0.8821,0.8967,0.9066,0.8967,0.9035,0.9051,0.8947,0.8965,0.8995,0.8868,0.8918,0.8814,0.8946,0.8840,0.8933,0.8891,0.9102,2,0.9051,2001-04-13
2001-04-09,0.9035,0.9051,0.8947,0.8965,0.90000,0.89861,0.898467,0.001073,0.006785,-0.001775,0.022495,0.006937,0.0070,0.0068,0.0099,0.0100,-1.0,90.0,117.2,-1,-1.0,1.0,-1.0,0.8966,0.9066,0.8966,0.9051,0.9091,0.9080,0.8927,0.8943,0.8937,0.9034,0.8967,0.9066,0.8964,0.8995,0.8868,0.8898,0.8918,0.8814,0.8946,0.8840,0.8933,0.8891,0.8895,0.8834,0.8895,1,0.8814,2001-04-16
2001-04-10,0.8964,0.8995,0.8868,0.8898,0.89310,0.89927,0.896674,0.004004,0.007279,-0.009483,0.003709,-0.001788,0.0066,0.0070,0.0068,0.0099,-1.0,89.2,117.6,-1,0.0,-1.0,1.0,0.9035,0.8966,0.9066,0.9051,0.9051,0.9091,0.8947,0.8927,0.8943,0.8965,0.9034,0.8967,0.8899,0.8918,0.8814,0.8884,0.8946,0.8840,0.8933,0.8891,0.8895,0.8834,0.8912,0.8780,0.8832,0,0.8780,2001-04-17


In [115]:
mb.print_marubozu_chart()

/Users/stuartdaw/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [116]:
mb.print_pattern_list()

Len: 64

list:
   pattern_end
0   2000-10-11
1   2000-10-20
2   2001-04-05
3   2001-04-09
4   2001-08-20
..         ...
59  2018-09-17
60  2018-10-22
61  2018-12-10
62  2019-01-31
63  2019-02-13

[64 rows x 1 columns]


In [117]:
daily_pattern = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/targets/daily_pattern.csv', 
                           parse_dates=True)

In [118]:
print(daily_pattern)

   pattern_end
0   2001-02-08
1   2001-04-04
2   2001-04-09
3   2001-08-20
4   2002-02-20
..         ...
57  2018-09-14
58  2018-10-22
59  2018-12-10
60  2019-01-31
61  2019-02-13

[62 rows x 1 columns]


In [119]:
mb.create_csv_pattern()

In [120]:
mb.create_csv_market()

In [121]:
mb.market_data.tail()

,open,high,low,close,mid,wk_mv_avg,mnth_mv_avg,volatility_3_day,volatility_10_day,pct_chge_3_prds,pct_chge_5_prds,pct_chge_10_prds,height,height-1,height-2,height-3,direction,gold_usd,gold_euro,marubozu,marubozu+1,marubozu-1,marubozu-2,day-1_open,day-2_open,day-3_open,day-1_high,day-2_high,day-3_high,day-1_low,day-2_low,day-3_low,day-1_close,day-2_close,day-3_close,day+1_open,day+1_high,day+1_low,day+1_close,day+2_high,day+2_low,day+3_high,day+3_low,day+4_high,day+4_low,day+5_high,day+5_low,exit_price,select,target,date+5
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-18,1.11312,1.11426,1.11105,1.11257,1.112845,1.114317,1.107608,0.000542,0.001949,-0.002098,0.001152,0.004463,0.00055,0.00162,0.00145,0.00383,-1.0,512.2,540.6,0,0.0,0.0,0.0,1.11473,1.11328,1.11710,1.11746,1.11581,1.11860,1.11293,1.11261,1.11120,1.11311,1.11473,1.11327,1.11256,1.11442,1.11072,1.11157,1.11236,1.10662,1.10958,1.10699,1.10934,1.10690,1.10953,1.10730,1.11202,0,1.10662,2019-12-25
2019-12-19,1.11256,1.11442,1.11072,1.11157,1.112065,1.113604,1.107830,0.000456,0.001945,-0.001741,-0.003196,0.002307,0.00099,0.00055,0.00162,0.00145,-1.0,513.1,542.0,0,0.0,0.0,0.0,1.11312,1.11473,1.11328,1.11426,1.11746,1.11581,1.11105,1.11293,1.11261,1.11257,1.11311,1.11473,1.11156,1.11236,1.10662,1.10783,1.10958,1.10699,1.10934,1.10690,1.10953,1.10730,1.11212,1.10821,1.11058,0,1.10662,2019-12-26
2019-12-20,1.11156,1.11236,1.10662,1.10783,1.109695,1.112506,1.107962,0.000761,0.002042,-0.003793,-0.004923,0.001358,0.00373,0.00099,0.00055,0.00162,-1.0,513.9,544.4,0,0.0,0.0,0.0,1.11256,1.11312,1.11473,1.11442,1.11426,1.11746,1.11072,1.11105,1.11293,1.11157,1.11257,1.11311,1.10783,1.10958,1.10699,1.10881,1.10934,1.10690,1.10953,1.10730,1.11212,1.10821,1.12108,1.11175,1.10410,0,1.10690,2019-12-27
2019-12-23,1.10783,1.10958,1.10699,1.10881,1.108320,1.111369,1.108154,0.000722,0.001986,-0.004066,-0.005103,0.001953,0.00098,0.00373,0.00099,0.00055,1.0,515.0,545.4,0,0.0,0.0,0.0,1.11156,1.11256,1.11312,1.11236,1.11442,1.11426,1.10662,1.11072,1.11105,1.10783,1.11157,1.11257,1.10880,1.10934,1.10690,1.10849,1.10953,1.10730,1.11212,1.10821,1.12108,1.11175,1.12207,1.11815,1.10979,0,1.12207,2019-12-30
2019-12-24,1.10880,1.10934,1.10690,1.10849,1.108645,1.110314,1.108480,0.001226,0.001935,-0.003075,-0.004736,0.000758,0.00031,0.00098,0.00373,0.00099,-1.0,515.0,545.3,0,0.0,0.0,0.0,1.10783,1.11156,1.11256,1.10958,1.11236,1.11442,1.10699,1.10662,1.11072,1.10881,1.10783,1.11157,1.10848,1.10953,1.10730,1.10892,1.11212,1.10821,1.12108,1.11175,1.12207,1.11815,1.12391,1.12020,1.10818,0,1.10730,2019-12-31


In [122]:
mb.print_pattern_list()

Len: 64

list:
   pattern_end
0   2000-10-11
1   2000-10-20
2   2001-04-05
3   2001-04-09
4   2001-08-20
..         ...
59  2018-09-17
60  2018-10-22
61  2018-12-10
62  2019-01-31
63  2019-02-13

[64 rows x 1 columns]


In [ ]:
\

In [ ]:
# Create Daily Data and remove weekend as doesnt have same liquidity and not 
# going to use to form patterns

daily = eur_usd.resample('B').agg({'open':'first','high':'max',
                                        'low':'min', 'close':'last',
                                        'year':'max','month':'max',
                                        'day':'first'})

daily['day_name'] = daily.index.day_name()
# daily.drop(daily.loc[(daily['day_name'] == 'Sunday') 
#                     | (daily['day_name'] == 'Saturday')].index, inplace=True)
daily.index

In [ ]:
daily.head(14)

In [ ]:
daily.isnull().sum()

In [ ]:
daily.loc[daily['open'].isnull()]

#### Check Dates and Remove Nan's

After checking these dates I noticed:

+ A lot of them were holidays - christmas, new years day
+ They were mostly in the first couple of years

I have decided to remove these for the follwoing reason.
+ The volatility and low liquidity could lead to false patterns
+ Most profiessional traders avoid the markets around holiday periods
+ The number of Nan's is relatively small compared to the dataset, so im prepared to lose some data and maintain accuracy.

[ECB Data](https://sdw.ecb.europa.eu/quickview.do?SERIES_KEY=120.EXR.D.USD.EUR.SP00.A)

[FOREX holidays](https://freshforex.com/analitics/holidays/)

In [ ]:
# Errors due to public holidays
daily.dropna(inplace=True)

In [ ]:
daily.isnull().sum()

In [ ]:
daily.index

In [ ]:
daily.shape

In [ ]:
# daily.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/daily.csv', index=True)

---

---

# 3.0 Pattern Matching

---

## 3.2 Daily Data

In [ ]:
daily.head()

In [ ]:
gold.head()

In [ ]:
# gold2 = gold.resample('B').max()

In [ ]:
daily.head()

In [ ]:
daily.isnull().sum()

In [ ]:
gold.shape, daily.shape

In [ ]:
# daily['gold_usd'] = gold['usd']
daily = pd.concat([daily, gold], axis=1)

In [ ]:
gold.index

In [ ]:
daily.head(5)

In [ ]:
daily.tail()

In [ ]:
daily.isnull().sum()

In [ ]:
daily.loc[daily['open'].isnull()]

In [ ]:
# drop na as they are extra from the gold data
daily.dropna(inplace=True)

In [ ]:
daily.isnull().sum()

# Loop through and find the patterns

In [ ]:
# Is Marubozu? Look for candlesticks with above a height and determine the timeframes direction

def is_marubozu(row, min_height= 0.005):
        
    # Check its a significant height
    if abs(row['open'] - row['close']) < min_height:
        return 0
    
    # Check the direction
    if row['open'] > row['close']:
        return -1
    elif row['open'] < row['close']:
        return 1
        
    return 0

In [ ]:
# Tag the line with information on the 
daily['marubozu'] = daily.apply(lambda row: is_marubozu(row), axis=1)

In [ ]:
daily['marubozu+1'] = daily['marubozu'].shift(-1)
daily['marubozu-1'] = daily['marubozu'].shift(1)
daily['marubozu-2'] = daily['marubozu'].shift(2)

In [ ]:
# See how many Marubozu have been created
daily['marubozu'].value_counts()

---

In [ ]:
daily['av_3_height'] = daily['height'].rolling(3).mean()

---

## Add Moving averages


In [ ]:
daily['wk_mv_av'] = daily['close'].rolling(7).mean()
daily['mnth_mv_av'] = daily['close'].rolling(30).mean()
daily['qtr_mv_av'] = daily['close'].rolling(120).mean()

In [ ]:
daily.head(34)

In [ ]:
daily.index

---

### Add Volatility

In [ ]:
daily['vol'] = daily['close'].pct_change().rolling(3).std()

## Create rows that match the pattern + add additional features

In [ ]:
#### Add previous 5 time frames
daily_pre = daily.drop(columns=['year','month','day','day_name'])

In [ ]:
daily_pre.index

In [ ]:
daily_pre.isnull().sum()

In [ ]:
daily_pre.head()

In [ ]:
daily_pre['day-1_open'] = daily_pre['open'].shift(1)
daily_pre['day-2_open'] = daily_pre['open'].shift(2)
daily_pre['day-3_open'] = daily_pre['open'].shift(3)

daily_pre['day-1_high'] = daily_pre['high'].shift(1)
daily_pre['day-2_high'] = daily_pre['high'].shift(2)
daily_pre['day-3_high'] = daily_pre['high'].shift(3)

daily_pre['day-1_low'] = daily_pre['low'].shift(1)
daily_pre['day-2_low'] = daily_pre['low'].shift(2)
daily_pre['day-3_low'] = daily_pre['low'].shift(3)

daily_pre['day-1_close'] = daily_pre['close'].shift(1)
daily_pre['day-2_close'] = daily_pre['close'].shift(2)
daily_pre['day-3_close'] = daily_pre['close'].shift(3)

daily_pre.head()

In [ ]:
# create future columns for target and matching connected Marubuzo
daily_pre['day+1_open'] = daily_pre['open'].shift(-1)
daily_pre['day+1_high'] = daily_pre['high'].shift(-1)
daily_pre['day+1_low'] = daily_pre['low'].shift(-1)
daily_pre['day+1_close'] = daily_pre['close'].shift(-1)

daily_pre['day+2_high'] = daily_pre['high'].shift(-2)
daily_pre['day+3_high'] = daily_pre['high'].shift(-3)
daily_pre['day+4_high'] = daily_pre['high'].shift(-4)
daily_pre['day+5_high'] = daily_pre['high'].shift(-5)

# Calculate the date of 5 business days ahead
daily_pre['date+5'] = daily_pre.index.shift(5, freq='B')

daily_pre.head(10)

In [ ]:
daily_pre['target'] = daily_pre[['day+1_high','day+2_high','day+3_high','day+4_high','day+5_high']].apply(max, axis=1)

In [ ]:
def choose_exit_price(row):
        return row['close'] + (row['height'] * 1)

In [ ]:
daily_pre['double_height'] = daily_pre.apply(choose_exit_price, axis=1)

In [ ]:
daily_pre.head()

In [ ]:
daily_pre.isnull().sum()

In [ ]:
daily_pre.dropna(inplace=True)

In [ ]:
daily_pre.isnull().sum()

In [ ]:
daily_pre.head(5)

In [ ]:
# features = daily_pre.loc[((daily_pre['marubozu'] == 1) &
#                          ((daily_pre['marubozu+1'] == -1) | (daily_pre['marubozu-1'] == -1))) |
#                          ((daily_pre['marubozu'] == -1) &
#                          ((daily_pre['marubozu+1'] == 1) | (daily_pre['marubozu-1'] == 1)))]

In [ ]:
# data = [[0,0,0,0,0], [1,-1,1,1,0],[-1,1,-1,1,0], [0,-1,1,1,1],[0,1,-1,1,1],[0,0,1,-1,1],[0,0,-1,1,1]] 
  
# # Create the pandas DataFrame 
# df = pd.DataFrame(data, columns = ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','expected']) 
# df

In [ ]:
# # Select rows
# def tag_rows(daily_pre):
    
#     if abs(daily_pre['marubozu']) != 1:
#         return 0
    
#     if (daily_pre['marubozu-2'] == 1 and daily_pre['marubozu-1'] == -1) or (daily_pre['marubozu-2'] == -1 and daily_pre['marubozu-1'] == 1):
#             return 0

#     if (daily_pre['marubozu-1'] == 1 and daily_pre['marubozu'] == -1) or (daily_pre['marubozu-1'] == -1 and daily_pre['marubozu'] == 1):
#             return 1
        
#     if (daily_pre['marubozu+1'] == 1 and daily_pre['marubozu'] == -1) or (daily_pre['marubozu+1'] == -1 and daily_pre['marubozu'] == 1):
#             return 1

In [ ]:
# data = [[0,0,0,0,0,0,0,0,0,0,0], [1,-1,1,1,0,0,0,0,0,0,0],[-1,1,-1,1,0,0,0,0,0,0,0], 
#         [0,-1,1,1,10,5,5,10,7,7,1],[0,1,-1,1,5,10,10,5,7,7,1],
#         [0,0,1,-1,10,5,7,7,5,10,1],[0,0,-1,1,5,10,7,7,10,5,1]]
  
# # Create the pandas DataFrame 
# df = pd.DataFrame(data, columns = ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','open','close',
#                                    'day-1_open','day-1_close','day+1_open','day+1_close','expected']) 
# df

In [ ]:
# Select rows that have a opposite pattern of very similar height.

def tag_rows(daily_pre, margin=0.0005):
    
    if abs(daily_pre['marubozu']) != 1:
         return 0

    if ((daily_pre['marubozu-2'] == 1 and daily_pre['marubozu-1'] == -1) or (
            daily_pre['marubozu-2'] == -1 and daily_pre['marubozu-1'] == 1)) and (
        ((abs(daily_pre['day-2_open'] - daily_pre['day-1_close']) < margin) and
        (abs(daily_pre['day-2_close'] - daily_pre['day-1_open']) < margin))):
            return 0

    if  ((daily_pre['marubozu-1'] == 1 and daily_pre['marubozu'] == -1) or (
        daily_pre['marubozu-1'] == -1 and daily_pre['marubozu'] == 1)) and (
        ((abs(daily_pre['open'] - daily_pre['day-1_close']) < margin) and
        (abs(daily_pre['close'] - daily_pre['day-1_open']) < margin))):
            return 1
        
    if ((daily_pre['marubozu+1'] == 1 and daily_pre['marubozu'] == -1) or (
        daily_pre['marubozu+1'] == -1 and daily_pre['marubozu'] == 1)) and (
        ((abs(daily_pre['open'] - daily_pre['day+1_close']) < margin) and
        (abs(daily_pre['close'] - daily_pre['day+1_open']) < margin))):
            return 1 

    return 0

In [ ]:
daily_pre['select'] = daily_pre.apply(lambda row: tag_rows(row), axis=1)

In [ ]:
daily_pre.loc[(daily_pre.index > '2004-04-1') & (daily_pre.index < '2004-04-11'),
              ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','open','close',
                                   'day-1_open','day-1_close','day+1_open','day+1_close','select']]

In [ ]:
features = daily_pre.loc[daily_pre['select'] == 1]

In [ ]:
features.head(10)

In [ ]:
features.shape

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=features.index,
                open=features['open'],
                high=features['high'],
                low=features['low'],
                close=features['close'])])

fig.add_trace(go.Line(x=daily.index, y=(daily['close']-daily['open'])+daily['open'], name='average',
                         line=dict(color='black', width=0.1, dash='dot')))

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [ ]:
# Store Dates for start point of predictions

In [ ]:
count = 0
dates = []
for row in features.itertuples():
    if count%2 == 0:
        dates.append(row[0])
    count+=1

In [ ]:
dates_daily = pd.DataFrame(dates)

In [ ]:
dates_daily.rename(columns={0:'pattern_end'}, inplace=True);

In [ ]:
dates_daily['pattern_end'] = pd.to_datetime(dates_daily['pattern_end'])

In [ ]:
# dates_daily.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/targets/daily_pattern.csv', index=False)

In [ ]:
# Create data using cut off dates

In [ ]:
# # Save the dataframes 
# for date in dates:
#     df = daily[(daily.index.get_level_values(0) >= '2000-05-30') & (
#     daily.index.get_level_values(0) <= date)]
#     df.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/targets/daily_'+ str(date) + '.csv', index=True)

In [ ]:
daily_pre.shape

In [ ]:
daily_pre.index

In [ ]:
# daily_pre.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/daily.csv', index=True)

---

---

## 3.1 Monthly Data

# Loop through and find the patterns

In [ ]:
# Is Marubozu? Look for candlesticks with above a height and determine the timeframes direction


def is_marubozu(row, min_height= 0.005):
        
    # Check its a significant height
    if abs(row['open'] - row['close']) < min_height:
        return 0
    
    # Check the direction
    if row['open'] > row['close']:
        return -1
    elif row['open'] < row['close']:
        return 1
        
    return 0

In [ ]:
# Tag the line with information on the 
monthly['marubozu'] = monthly.apply(lambda row: is_marubozu(row), axis=1)

In [ ]:
monthly['marubozu+1'] = monthly['marubozu'].shift(-1)
monthly['marubozu-1'] = monthly['marubozu'].shift(1)
monthly['marubozu-2'] = monthly['marubozu'].shift(2)

In [ ]:
# See how many Marubozu have been created
monthly['marubozu'].value_counts()

In [ ]:
monthly.isnull().sum()

---

## add height

In [ ]:
monthly['height'] = monthly['close'] - monthly['open']

In [ ]:
monthly.head(20)

In [ ]:
monthly['av_3_height'] = monthly['height'].rolling(3).mean()

In [ ]:
monthly.head(10)

---

## Add Moving averages


In [ ]:
monthly['mnth_mv_av'] = monthly['close'].rolling(2).mean()
monthly['qtr_mv_av'] = monthly['close'].rolling(3).mean()
monthly['yr_mv_av'] = monthly['close'].rolling(12).mean()


In [ ]:
monthly.head(13)

## Create rows that match the pattern + add additional features

In [ ]:
#### Add previous 5 time frames
monthly = monthly.drop(columns=['year','month'])

In [ ]:
monthly.head()

In [ ]:
monthly['mth-1_open'] = monthly['open'].shift(1)
monthly['mth-2_open'] = monthly['open'].shift(2)
monthly['mth-3_open'] = monthly['open'].shift(3)

monthly['mth-1_high'] = monthly['high'].shift(1)
monthly['mth-2_high'] = monthly['high'].shift(2)
monthly['mth-3_high'] = monthly['high'].shift(3)

monthly['mth-1_low'] = monthly['low'].shift(1)
monthly['mth-2_low'] = monthly['low'].shift(2)
monthly['mth-3_low'] = monthly['low'].shift(3)

monthly['mth-1_close'] = monthly['close'].shift(1)
monthly['mth-2_close'] = monthly['close'].shift(2)
monthly['mth-3_close'] = monthly['close'].shift(3)

monthly.head()

In [ ]:
# create future columns for target and matching connected Marubuzo
monthly['mth+1_open'] = monthly['open'].shift(-1)
monthly['mth+1_high'] = monthly['high'].shift(-1)
monthly['mth+1_low'] = monthly['low'].shift(-1)
monthly['mth+1_close'] = monthly['close'].shift(-1)

monthly['mth+2_high'] = monthly['high'].shift(-2)
monthly['mth+3_high'] = monthly['high'].shift(-3)
monthly.head()

In [ ]:
monthly['target'] = monthly[['mth+1_high','mth+2_high','mth+3_high']].apply(max, axis=1)

In [ ]:
monthly.head()

In [ ]:
monthly.head(5)

In [ ]:
# features = daily_pre.loc[((daily_pre['marubozu'] == 1) &
#                          ((daily_pre['marubozu+1'] == -1) | (daily_pre['marubozu-1'] == -1))) |
#                          ((daily_pre['marubozu'] == -1) &
#                          ((daily_pre['marubozu+1'] == 1) | (daily_pre['marubozu-1'] == 1)))]

In [ ]:
# data = [[0,0,0,0,0], [1,-1,1,1,0],[-1,1,-1,1,0], [0,-1,1,1,1],[0,1,-1,1,1],[0,0,1,-1,1],[0,0,-1,1,1]] 
  
# # Create the pandas DataFrame 
# df = pd.DataFrame(data, columns = ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','expected']) 
# df

In [ ]:
# # Select rows
# def tag_rows(daily_pre):
    
#     if abs(daily_pre['marubozu']) != 1:
#         return 0
    
#     if (daily_pre['marubozu-2'] == 1 and daily_pre['marubozu-1'] == -1) or (daily_pre['marubozu-2'] == -1 and daily_pre['marubozu-1'] == 1):
#             return 0

#     if (daily_pre['marubozu-1'] == 1 and daily_pre['marubozu'] == -1) or (daily_pre['marubozu-1'] == -1 and daily_pre['marubozu'] == 1):
#             return 1
        
#     if (daily_pre['marubozu+1'] == 1 and daily_pre['marubozu'] == -1) or (daily_pre['marubozu+1'] == -1 and daily_pre['marubozu'] == 1):
#             return 1

In [ ]:
# data = [[0,0,0,0,0,0,0,0,0,0,0], [1,-1,1,1,0,0,0,0,0,0,0],[-1,1,-1,1,0,0,0,0,0,0,0], 
#         [0,-1,1,1,10,5,5,10,7,7,1],[0,1,-1,1,5,10,10,5,7,7,1],
#         [0,0,1,-1,10,5,7,7,5,10,1],[0,0,-1,1,5,10,7,7,10,5,1]]
  
# # Create the pandas DataFrame 
# df = pd.DataFrame(data, columns = ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','open','close',
#                                    'day-1_open','day-1_close','day+1_open','day+1_close','expected']) 
# df

In [ ]:
# Select rows that have a opposite pattern of very similar height.

def tag_rows(df, margin=0.003):
    
    if abs(df['marubozu']) != 1:
         return 0

    if ((df['marubozu-2'] == 1 and df['marubozu-1'] == -1) or (
            df['marubozu-2'] == -1 and df['marubozu-1'] == 1)) and (
        ((abs(df['mth-2_open'] - df['mth-1_close']) < margin) and
        (abs(df['mth-2_close'] - df['mth-1_open']) < margin))):
            return 0

    if  ((df['marubozu-1'] == 1 and df['marubozu'] == -1) or (
        df['marubozu-1'] == -1 and df['marubozu'] == 1)) and (
        ((abs(df['open'] - df['mth-1_close']) < margin) and
        (abs(df['close'] - df['mth-1_open']) < margin))):
            return 1
        
    if ((df['marubozu+1'] == 1 and df['marubozu'] == -1) or (
        df['marubozu+1'] == -1 and df['marubozu'] == 1)) and (
        ((abs(df['open'] - df['mth+1_close']) < margin) and
        (abs(df['close'] - df['mth+1_open']) < margin))):
            return 1 

    return 0

In [ ]:
monthly['select'] = monthly.apply(lambda row: tag_rows(row), axis=1)

In [ ]:
monthly.loc[(monthly.index > '2004-01-1') & (monthly.index < '2004-12-31'),
              ['marubozu-2', 'marubozu-1','marubozu','marubozu+1','open','close',
                                   'mth-1_open','mth-1_close','mth+1_open','mth+1_close','select']]

In [ ]:
features_mth = monthly.loc[monthly['select'] == 1]

In [ ]:
features_mth.head(10)

In [ ]:
features_mth.shape

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=features_mth.index,
                open=features_mth['open'],
                high=features_mth['high'],
                low=features_mth['low'],
                close=features_mth['close'])])

fig.add_trace(go.Line(x=daily.index, y=daily['close'], name='average',
                         line=dict(color='black', width=0.1, dash='dot')))

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [ ]:
# monthly.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/monthly.csv', index=True)

In [ ]:
# Store Dates for start point of predictions

In [ ]:
# count = 0
# dates = []
# for row in features_mth.itertuples():
#     if count%2 == 0:
#         dates.append(row[0])
#     count+=1

In [ ]:
# Create data using cut off dates

In [ ]:
# Save the dataframes 
# for date in dates:
#     df = monthly[(monthly.index.get_level_values(0) >= '2000-05-30') & (
#     daily.index.get_level_values(0) <= date)]
#     df.to_csv('/Users/stuartdaw/Documents/Capstone_data/data/targets/monthly_'+ str(date) + '.csv', index=True)